Since the thomas_id in the legislators data is filtered from the preceeding zero's we have to do the same.

In [1]:
def Tidclean(Tid):
    i = 0
    while Tid[i] == "0":
        i += 1
    return Tid[i:]

First we will create another csv file which will make it easier to look up the correlated party with the thomas or bioguide id. 

In [2]:
import csv
with open('data/legislators/legislators-current.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    bioguide_index = your_list[0].index("bioguide_id")
    thomas_index = your_list[0].index("thomas_id")
    party_index = your_list[0].index("party")
    legis_current = []
    for row in range(len(your_list)):
        person = your_list[row]
        if len(person[thomas_index]) == 5:
            thomas_id = Tidclean(person[thomas_index])
            legis_current.append([person[bioguide_index], thomas_id, person[party_index]])
        else:
            legis_current.append([person[bioguide_index], person[thomas_index], person[party_index]])

In [3]:
import csv
with open('data/legislators/legislators-historical.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    bioguide_index = your_list[0].index("bioguide_id")
    thomas_index = your_list[0].index("thomas_id")
    party_index = your_list[0].index("party")
    legis_historical = []
    for row in range(1,len(your_list)):
        person = your_list[row]
        if len(person[thomas_index]) == 5:
            thomas_id = Tidclean(person[thomas_index])
            legis_historical.append([person[bioguide_index], thomas_id, person[party_index]])
        else:
            legis_historical.append([person[bioguide_index], person[thomas_index], person[party_index]])

In [4]:
legislators_ID = []
legislators_ID.extend(legis_current)
legislators_ID.extend(legis_historical)

In [5]:
import csv
with open('data/legislators/legislators_ID.csv', 'w') as outcsv:   
    writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    for item in legislators_ID:
        writer.writerow([item[0], item[1], item[2]])

In [6]:
with open('data/legislators/legislators_ID.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_bioguide = {}
    dict_thomas = {}
    for person in reader:
        dict_bioguide[person[0]] = person[2]
        dict_thomas[person[1]] = person[2]

In [7]:
# first we do is for hr bills, congress nr 112
import json

lo_bills = []

# This is our module to get all the bill file paths.
from bill_paths import get_bill_file_paths 

# All the bills are organized alphabetically so instead of hr1, hr2, hr3 you'll get hr1, hr10, hr100 but the order is not relevant to us.
bill_paths = get_bill_file_paths(112)

#allyears = range(1, 117) #Current year is 116
#bill_paths = {}
#for year in allyears:
 #   bill_paths[year] = get_bill_file_paths(year) 
# if year 

for path in bill_paths:
    with open(path) as json_file:
        bill = json.load(json_file)
    # First check if the bill contains all the items we want for our project.
        if ("bill_id" and "official_title" and "subject_top_term" and "summary" and "sponsor") in bill.keys():
            if bill["summary"] != None:
                dictionary = {}
                dictionary["bill_id"] = bill["bill_id"]
    # We use the official title 
                dictionary["title"] = bill["official_title"] 
    # Most bills have many subjects so we decid to only look at the top term for clarity
                dictionary["subject"] = bill["subjects_top_term"]
                summary_ = bill["summary"]
                dictionary["summary"] = summary_["text"]
                sponsor = bill["sponsor"]
                # find party in csv file from ID
                if "thomas_id" in sponsor.keys():
                    thomas_id = sponsor["thomas_id"]
                    party = dict_thomas[Tidclean(thomas_id)]
                if "bioguide_id" in sponsor.keys():
                    party = dict_bioguide[sponsor["bioguide_id"]]
                dictionary["party"] = party
    
    # Appended to the list of bills 
        lo_bills.append(dictionary)
            
#per bill we create a dictionary containing the following data:
#Bill id, Title, Summary
# then, we create a list of all these dictionaries

In [8]:
import csv

with open('data/112.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter='|')
    filewriter.writerow(['bill_id', 'title', 'subject','summary', 'party'])
    for bill in lo_bills:
        filewriter.writerow([bill['bill_id'], bill['title'], bill['subject'], bill['summary'], bill['party']])